# Linear Regression with PyTorch
```We'll create/build a linear regression model that predicts crop yields for apples and oranges (target variables) by looking at the average temperature, rainfall and humidity (input variables or features) in a region.``` <br>Here's the training data:
![Data Set](6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :
###### This is known as linear regression:
`yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
 yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
`
<br>
Here w11,w12,w13,w21,w22,w23 i.e., "weight" are simply numbers,<br>
Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

In [1]:
import numpy as np
import torch

# Training data:

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([
    [73, 67, 43],
    [91, 88, 64],
    [87, 134, 58],
    [102, 43, 37],
    [69, 96, 70]
],dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([
    [56, 70],
    [81, 101],
    [119, 133],
    [22, 37],
    [103, 119]
],dtype='float32')

In [4]:
# Converting inputs and targets to tenosrs
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# Linear regression model from scratch
The weights and biases (w11, w12,... w23, b1 & b2) can also be represented as matrices, initialized as random values. The first row of w and the first element of b are used to predict the first target variable i.e. yield of apples, and similarly the second for oranges.

In [5]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.2472,  0.5888,  3.8004],
        [-1.1537,  0.2929,  1.3149]], requires_grad=True)
tensor([-0.2872,  0.6040], requires_grad=True)


# Now we'll start creating the model<br>
Our model is simply a function that performs a matrix multiplication of the inputs and the weights w (transposed) and adds the bias b (replicated for each observation).
![Model](WGXLFvA.png)

In [6]:
def model(x):
    return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t()` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [7]:
# Lets Generate some predictions
preds = model(inputs)

tensor([[184.5342,  -7.4474],
        [272.2579,   5.5511],
        [277.5299,  15.7502],
        [140.4312, -55.8237],
        [305.2094,  41.1649]], grad_fn=<AddBackward0>)


# Lets compare the `preds` with `targets`

In [8]:
print('Predictions: \n',preds)
print('Tragets: \n',targets)

Predictions: 
 tensor([[184.5342,  -7.4474],
        [272.2579,   5.5511],
        [277.5299,  15.7502],
        [140.4312, -55.8237],
        [305.2094,  41.1649]], grad_fn=<AddBackward0>)
Tragets: 
 tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


###### We can see that the preds is pretty far of the targets so now what to do:

# Loss function
Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method:

`1.`Calculate the difference between the two matrices (preds and targets).<br>
`2.`Square all elements of the difference matrix to remove negative values.<br>
`3.`Calculate the average of the elements in the resulting matrix.
<br>**The result is a single number, known as the mean squared error (MSE).**

In [9]:
# Defining MSE loss function
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff*diff)/diff.numel()

`torch.sum` returns the sum of all the elements in a tensor, and the `.numel` method returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [10]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(17667.7598, grad_fn=<DivBackward0>)


# Lets Improve our model by
gradient descent method

# Compute gradients
With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have `requires_grad` set to `True`.

In [11]:
# Computing gradients
loss.backward()

In [12]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.2472,  0.5888,  3.8004],
        [-1.1537,  0.2929,  1.3149]], requires_grad=True)
tensor([[13322.3994, 14238.0273,  9099.7637],
        [-7875.7773, -8152.7100, -5024.4785]])


Before we proceed, we reset the gradients to zero by calling `.zero_()` method. 
We need to do this, because PyTorch accumulates, 
gradients i.e. the next time we call `.backward` on the loss,
the new gradient values will get added to the existing gradient values, 
which may lead to unexpected results.

In [13]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


# Optimization Algorithm
## Adjust weights and biases using gradient descent
We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

`1.`Generate predictions

`2.`Calculate the loss

`3.`Compute gradients w.r.t the weights and biases

`4.`Adjust the weights by subtracting a small quantity proportional to the gradient

`5.`Reset the gradients to zero

Let's implement the above step by step.

In [14]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[184.5342,  -7.4474],
        [272.2579,   5.5511],
        [277.5299,  15.7502],
        [140.4312, -55.8237],
        [305.2094,  41.1649]], grad_fn=<AddBackward0>)


In [15]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(17667.7598, grad_fn=<DivBackward0>)


In [16]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[13322.3994, 14238.0273,  9099.7637],
        [-7875.7773, -8152.7100, -5024.4785]])
tensor([159.7925, -92.1610])


In [17]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [18]:
print(w)
print(b)

tensor([[-0.3804,  0.4464,  3.7094],
        [-1.0749,  0.3744,  1.3652]], requires_grad=True)
tensor([-0.2888,  0.6049], requires_grad=True)


In [20]:
# Calculate Loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(12052.0928, grad_fn=<DivBackward0>)


# Train for multiple epocs

In [43]:
# Train for 100 epochs
for i in range(10000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [45]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(0.5230, grad_fn=<DivBackward0>)


In [46]:
# Lets check
print('Prediction: \n',preds)
print('Targets: \n',targets)

Prediction: 
 tensor([[ 57.1131,  70.4102],
        [ 82.2401, 100.6072],
        [118.7003, 132.9409],
        [ 21.0878,  37.0023],
        [101.9132, 119.1551]], grad_fn=<AddBackward0>)
Targets: 
 tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [47]:
import jovian
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "manishshah120/02-linear-regression-7d6ef" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/manishshah120/02-linear-regression-7d6ef


'https://jovian.ml/manishshah120/02-linear-regression-7d6ef'

# -------The End------

# linear regression using PyTorch built-ins
The model and training process above were implemented using basic matrix operations. But since this such a common pattern , PyTorch has several built-in functions and classes to make it easy to create and train models.

Let's begin by importing the torch.nn package from PyTorch, which contains utility classes for building neural networks.

In [48]:
import torch.nn as nn

In [49]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')
#Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

# Dataset and DataLoader
We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [50]:
from torch.utils.data import TensorDataset

In [51]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [52]:
from torch.utils.data import DataLoader

In [53]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [57]:
for xb, yb in train_dl:
    #print('Batch: ')
    print(xb)
    print(yb)
    break

tensor([[ 69.,  96.,  70.],
        [ 91.,  88.,  64.],
        [ 91.,  88.,  64.],
        [ 69.,  96.,  70.],
        [ 87., 134.,  58.]])
tensor([[103., 119.],
        [ 81., 101.],
        [ 81., 101.],
        [103., 119.],
        [119., 133.]])


# nn.Linear
Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [58]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.0156,  0.1203,  0.4094],
        [-0.1582,  0.0469, -0.5257]], requires_grad=True)
Parameter containing:
tensor([ 0.1513, -0.0278], requires_grad=True)


In [60]:
# Paramters
list(model.parameters())

[Parameter containing:
 tensor([[-0.0156,  0.1203,  0.4094],
         [-0.1582,  0.0469, -0.5257]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1513, -0.0278], requires_grad=True)]

In [62]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 24.6724, -31.0450],
        [ 35.5139, -43.9494],
        [ 38.6550, -38.0053],
        [ 18.8749, -33.6046],
        [ 39.2767, -43.2476],
        [ 24.6724, -31.0450],
        [ 35.5139, -43.9494],
        [ 38.6550, -38.0053],
        [ 18.8749, -33.6046],
        [ 39.2767, -43.2476],
        [ 24.6724, -31.0450],
        [ 35.5139, -43.9494],
        [ 38.6550, -38.0053],
        [ 18.8749, -33.6046],
        [ 39.2767, -43.2476]], grad_fn=<AddmmBackward>)

# Loss Function
instead of defining a loss function manually,  we can use the built-in loss function `mse_loss`

In [65]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities.


In [66]:
# Define loss function
loss_fn = F.mse_loss

In [67]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(10534.8691, grad_fn=<MseLossBackward>)


# Optimizer
Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD stands for `stochastic gradient descent`. It is called `stochastic` because samples are selected in batches (often with random shuffling) instead of as a single group.

In [68]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

# Train the model

In [86]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
 
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [87]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 1.6618
Epoch [20/100], Loss: 3.3960
Epoch [30/100], Loss: 2.3041
Epoch [40/100], Loss: 0.9514
Epoch [50/100], Loss: 2.0747
Epoch [60/100], Loss: 1.7751
Epoch [70/100], Loss: 2.2578
Epoch [80/100], Loss: 1.8934
Epoch [90/100], Loss: 2.1985
Epoch [100/100], Loss: 1.8031


In [88]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.2190,  70.3935],
        [ 82.1471,  99.6345],
        [118.8704, 134.9798],
        [ 21.1387,  37.5738],
        [101.8032, 117.0205],
        [ 57.2190,  70.3935],
        [ 82.1471,  99.6345],
        [118.8704, 134.9798],
        [ 21.1387,  37.5738],
        [101.8032, 117.0205],
        [ 57.2190,  70.3935],
        [ 82.1471,  99.6345],
        [118.8704, 134.9798],
        [ 21.1387,  37.5738],
        [101.8032, 117.0205]], grad_fn=<AddmmBackward>)

In [89]:
# Lets compare them with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

# Further Reading
We've covered a lot of ground this this tutorial, including *linear regression* and the *gradient descent* optimization algorithm. Here are a few resources if you'd like to dig deeper into these topics:

* For a more detailed explanation of derivates and gradient descent, see [these notes from a Udacity course](https://storage.googleapis.com/supplemental_media/udacityu/315142919/Gradient%20Descent.pdf). 

* For an animated visualization of how linear regression works, [see this post](https://hackernoon.com/visualizing-linear-regression-with-pytorch-9261f49edb09).

* For a more mathematical treatment of matrix calculus, linear regression and gradient descent, you should check out [Andrew Ng's excellent course notes](https://github.com/Cleo-Stanford-CS/CS229_Notes/blob/master/lectures/cs229-notes1.pdf) from CS229 at Stanford University.

* To practice and test your skills, you can participate in the [Boston Housing Price Prediction](https://www.kaggle.com/c/boston-housing) competition on Kaggle, a website that hosts data science competitions.

# Created this file while learning from Jovian.ml instructer Aakash N S

**The End**